# Test alignment procedure
This file is used to test the alignment procedure using the center of the fiber core as a reference and the reflected laser beam.

In [ ]:
from time import sleep
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import cv2

import matplotlib

matplotlib.rc('figure', figsize=(20, 10))

In [ ]:
from dispertech.models.electronics.arduino import ArduinoModel
from dispertech.models.cameras.basler import Camera
from experimentor import Q_
from experimentor.models.decorators import make_async_thread

In [ ]:
ard = ArduinoModel(0)
ard.initialize()
ard.laser_power(25)

In [ ]:
cam = Camera('da')
cam.initialize()

In [ ]:
# ard.move_mirror(0, 0, 2)
cam.set_exposure(Q_('.01ms'))
cam.trigger_camera()
img = cam.read_camera()[0]
mask = img>1000
M = cv2.moments(mask*img)
cx = M['m10']/M['m00']
cy = M["m01"] / M["m00"]
print(cx, cy)
fig, ax = plt.subplots(1)
ax.imshow(mask.T*img.T)
circ = Circle((cy, cx), radius=15,color='red')
ax.add_patch(circ)

In [ ]:
plt.hist(img.reshape(img.size,1))

In [ ]:
data = []
max_data_points = 10
accumulations_per_position = 50
class Empty: pass
e = Empty()
@make_async_thread
def acquire_data(e):
    print('Starting...')
    while True:
        try:
            data.append(([],[]))
            for i in range(accumulations_per_position):
                cam.trigger_camera()
                img = cam.read_camera()[0]
                if i==0: continue
                mask = img>1000
                M = cv2.moments(mask*img)
                cx = M['m10']/M['m00']
                cy = M["m01"] / M["m00"]
                data[-1][0].append(cx)
                data[-1][1].append(cy)
            ard.move_mirror(5, 0, 1)
            print(f'Finished position # {len(data)}')
            print(f'Average position: ({np.mean(data[-1][0])}, {np.mean(data[-1][1])})')
            if max_data_points and len(data) == max_data_points:
                break
        except KeyboardInterrupt:
            break
    print('Moving back')
    for i in range(len(max_data_points)):
        try:
            data.append(([],[]))
            for i in range(accumulations_per_position):
                cam.trigger_camera()
                img = cam.read_camera()[0]
                if i==0: continue
                mask = img>1000
                M = cv2.moments(mask*img)
                cx = M['m10']/M['m00']
                cy = M["m01"] / M["m00"]
                data[-1][0].append(cx)
                data[-1][1].append(cy)
            ard.move_mirror(5, 1, 1)
            print(f'Finished position # {len(data)}')
            print(f'Average position: ({np.mean(data[-1][0])}, {np.mean(data[-1][1])})')
        except KeyboardInterrupt:
            break
acquire_data(e)

In [ ]:
fig, ax = plt.subplots(2, figsize=(20,20))

dt = np.array(data)
C = np.zeros((2,len(data)))
for i in range(len(data)):
    ax[0].plot(data[i][0],data[i][1], 'o')
    C[0][i] = np.mean(data[i][0])
    C[1][i] = np.mean(data[i][1])

ax[1].plot(C[0,:],C[1][:], '.-')
ax[0].set_aspect('equal')
ax[1].set_aspect('equal')